## Whoscored 크롤링

## 만들기 영차영차

### 1. 기본 요약(점유율+ 득점 ..)  OK

In [52]:
# 다른 탭으로 옮겨서 정보를 가져오려면 클릭을 해야한다
# action chain 을 사용해서 클릭하고 정보 가져오는 방식으로 만들어보자
# 재미있을 것이다...! 기대된다

# 라이프치히 전술에 대한 분석, 그것을 위한 데이터를 크롤링한다.

# 분데스리가 1부리그 통합 통계비교. 라이프치히가 상위권,하위권인 항목을 골라서 가져올 것. 특정 항목에 대한 리스트를 만들어서 그것들만 가져오자...
# https://www.whoscored.com/Regions/81/Tournaments/3/Seasons/7405/Stages/16427/TeamStatistics/Germany-Bundesliga-2018-2019

# 라이프치히 + 모든 구단별로 강점,약점 리스트 크롤링해오기(재미로)


import requests
from bs4 import BeautifulSoup
from pprint import pprint
from selenium import webdriver
from pprint import pprint
import pandas as pd
from time import sleep


# 크롬창 열기
driver_path = 'C:\\SEONMIN\\chromedriver_win32\\chromedriver'
driver = webdriver.Chrome(driver_path)
driver.get('https://www.whoscored.com/Regions/81/Tournaments/3/Seasons/7405/Stages/16427/TeamStatistics/Germany-Bundesliga-2018-2019')



# 1. 기본 요약(점유율+ 득점 ..)  

init_df = pd.DataFrame([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])
count = 1

for c1 in range(1,9):
    col = []
    val =[]
    tt = driver.find_element_by_xpath('//*[@id="top-team-stats-summary-grid"]/thead/tr/th['+str(c1)+']').text
#     print(tt)
    col.append(tt)
    
    for a in range(1,19):
#         for b in range(1,19):
        summary_r_elem = driver.find_element_by_xpath('//*[@id="top-team-stats-summary-content"]/tr['+str(a)+']/td['+str(count)+']').text
        val.append(summary_r_elem)
        
#     print(val)
    count += 1
    temp_dict = {col[0]:val}
    temp_df = pd.DataFrame(temp_dict)
    init_df = init_df.join(temp_df)
#     print(init_df)
    
init_df = init_df.drop([0], axis = 1)    
init_df = init_df.set_index('R')
print(init_df)
init_df.to_csv('Whoscored_bundes_1819_Summary.csv')        


                      Team Goals Shots pg Discipline Possession% Pass%  \
R                                                                        
1            Bayern Munich    88     18.7        402        62.2  87.5   
2   RasenBallsport Leipzig    63     14.6        611        49.5  75.3   
3        Borussia Dortmund    81     13.2        403        56.8  86.0   
4         Bayer Leverkusen    69     15.2        501        57.7  84.4   
5               Hoffenheim    70       18        652        52.8  82.0   
6      Eintracht Frankfurt    60     12.4        563        47.6  73.1   
7                Wolfsburg    62     12.5        620        52.5  78.6   
8      Borussia M.Gladbach    55     13.5        430        52.3  84.3   
9            Werder Bremen    58     13.9        583        51.2  80.0   
10           Hertha Berlin    49     11.9        615        48.6  78.4   
11                Mainz 05    46     14.6        560        46.9  77.1   
12     Fortuna Duesseldorf    49     1

## 2. 디테일 가져오기  OK

## 함수로 정의하기

In [53]:
"""

2. 디테일
1) 태클(디폴트) t
2) 파울(디폴트) f 
3) 디스포제션(디폴트) p
4) 공중(디폴트) a
5) 패스-렝스(디폴트) pp
6) 키패스-렝스(디폴트) k


"""
import requests
from bs4 import BeautifulSoup
from pprint import pprint
from selenium import webdriver
from pprint import pprint
import pandas as pd
from time import sleep
from selenium.webdriver.common.action_chains import ActionChains

driver_path = 'C:\\SEONMIN\\chromedriver_win32\\chromedriver'
url = 'https://www.whoscored.com/Regions/81/Tournaments/3/Seasons/7405/Stages/16427/TeamStatistics/Germany-Bundesliga-2018-2019'


def get_whoscored():
    which = input('어떤 자료를 가져올까요? \n태클(T), 파울(F), 소유권 상실(P), 공중볼 경합(A), 패스(PP), 키패스(K)')
    driver = webdriver.Chrome(driver_path)
    driver.get(url)
    
    # 액션체인 사용
    actions = ActionChains(driver)
    
    # detail버튼 누르기
    detail_btn = driver.find_element_by_xpath('//*[@id="stage-team-stats-options"]/li[4]/a')    
    actions.move_to_element(detail_btn)
    actions.click(detail_btn)
    actions.perform()
    actions.reset_actions()
    
    # 카테고리 선택하기
    detail_category = driver.find_element_by_xpath('//*[@id="category"]')
    actions.move_to_element(detail_category)
    actions.click(detail_category)
    sleep(2)
    actions.send_keys(which)
    actions.perform()
    actions.reset_actions()

    # 선택한 카테고리로 이동하기
    detail_category = driver.find_element_by_xpath('//*[@id="category"]')
    actions.move_to_element(detail_category)
    actions.click(detail_category)
    actions.perform()
    actions.reset_actions()

    # 2초 대기해서 자료 기다리기
    sleep(2)
    
    ####################################################  
    ###
    ###   columns 그리고 본문을 리스트로 -> df 만들기 
    ###
    ####################################################



    ####### column list 만들기  ########
    col_table = driver.find_elements_by_tag_name('thead')[1].text
    # print(col_table)
    col_list = col_table.split(' ')
    # print(col_list)
    # print('*'*10)

    ###### 본문 list 만들기  #######  
    text_list = []
    real_table = driver.find_elements_by_tag_name('tbody')[2].text
    # print(real_table)
    real_table = real_table.split('\n')
    # print(real_table)

    for line in real_table:
        line_list =line.split(' ')
    #     print(line_list)
        if len(line_list) == (len(col_list)+1):
            double_name = line_list[1]+' '+line_list[2]
            line_list.insert(1,double_name)
    #         print(line_list)
            line_list.pop(2)
            line_list.pop(2)
    #         print(line_list)
            text_list.append(line_list)
        else:
            text_list.append(line_list)

    # print(text_list)
    temp_df = pd.DataFrame(text_list, columns=col_list)
    temp_df = temp_df.set_index('R')
    print(temp_df)
    return temp_df.to_csv('Whoscored_bundes_1819_'+which+'.csv') #, encoding='cp949')

In [54]:
get_whoscored()

어떤 자료를 가져올까요? 
태클(T), 파울(F), 소유권 상실(P), 공중볼 경합(A), 패스(PP), 키패스(K)a
                      Team Total   Won  Lost Rating
R                                                  
1            Bayern Munich  29.1  15.7  13.4   7.03
2   RasenBallsport Leipzig    41  22.4  18.6   6.96
3        Borussia Dortmund  28.2    14  14.2   6.93
4         Bayer Leverkusen  33.4  16.6  16.9   6.88
5               Hoffenheim  32.2  15.9  16.3   6.87
6      Eintracht Frankfurt  46.2  24.6  21.6   6.83
7                Wolfsburg  36.4  18.9  17.5   6.81
8      Borussia M.Gladbach  29.6  13.2  16.4   6.76
9            Werder Bremen  38.1  18.1  20.1   6.75
10           Hertha Berlin  40.4  20.9  19.5   6.74
11                Mainz 05  38.8  19.6  19.2   6.70
12     Fortuna Duesseldorf  36.9  18.7  18.2   6.67
13                Augsburg  40.9    20  20.9   6.65
14              Schalke 04    35    18    17   6.65
15                Freiburg  37.1  16.9  20.2   6.64
16             Hannover 96  40.2  18.6  21.6   6.

## 3. Bundesliga Positional Statistics
액션존 

In [55]:
import requests
from bs4 import BeautifulSoup
from pprint import pprint
from selenium import webdriver
from pprint import pprint
import pandas as pd
from time import sleep
from selenium.webdriver.common.action_chains import ActionChains


driver_path = 'C:\\SEONMIN\\chromedriver_win32\\chromedriver'
url = 'https://www.whoscored.com/Regions/81/Tournaments/3/Seasons/7405/Stages/16427/TeamStatistics/Germany-Bundesliga-2018-2019'

def get_position():
    print('1. Bundesliga 18-19 Bundesliga Positional Statistics')
    driver = webdriver.Chrome(driver_path)
    driver.get(url)
    
    # 액션체인 사용
    actions = ActionChains(driver)
    
    # action zone 버튼 누르기
    az_btn = driver.find_element_by_xpath('//*[@id="stage-pitch-stats-options"]/li[4]/a')    
    actions.move_to_element(az_btn)
    actions.click(az_btn)
    actions.perform()
    actions.reset_actions()
    sleep(1)
    
    # 정렬하기
    az_sort = driver.find_element_by_xpath('//*[@id="stage-touch-zones-grid"]/thead/tr/th[5]')
    actions.move_to_element(az_sort)
    actions.click(az_sort)
    actions.click(az_sort)
    actions.perform()
    actions.reset_actions()
    
    # 2초 대기해서 자료 기다리기
    sleep(2)
    
    
    #######################################################
    
    
    ####### column list 만들기  ########
    col_table = driver.find_elements_by_tag_name('thead')[7].text
#     print(col_table)
    col_list = col_table.split(' ')
    
    own_third = col_list[2]+' '+col_list[3]
    middle_third = col_list[4]+' '+col_list[5]
    oppo_third = col_list[6]+' '+col_list[7]
    col_list = [col_list[0],col_list[1], own_third, middle_third, oppo_third]
#     print(col_list)
#     print('*'*10)
    

    ###### 본문 list 만들기  #######  
    text_list = []
    real_table = driver.find_elements_by_tag_name('tbody')[8].text
#     print(real_table)
    real_table = real_table.split('\n')
#     print(real_table)

    for line in real_table:
        line_list =line.split(' ')
#         print(line_list)
        if len(line_list) == (len(col_list)+1):
            double_name = line_list[1]+' '+line_list[2]
            line_list.insert(1,double_name)
#             print(line_list)
            line_list.pop(2)
            line_list.pop(2)
#             print(line_list)
            text_list.append(line_list)
        else:
            text_list.append(line_list)

#     print(text_list)
    temp_df = pd.DataFrame(text_list, columns=col_list)
    temp_df = temp_df.set_index('R')
    print(temp_df)
    return temp_df.to_csv('Whoscored_bundes_1819_Positional Statistics.csv')

In [56]:
get_position()

1. Bundesliga 18-19 Bundesliga Positional Statistics
                      Team Own Third Middle Third Opposition Third
R                                                                 
1            Bayern Munich       21%          45%              34%
2   RasenBallsport Leipzig       22%          47%              32%
3      Eintracht Frankfurt       26%          46%              29%
4                 Mainz 05       26%          45%              29%
5               Hoffenheim       27%          45%              28%
6               Schalke 04       26%          46%              28%
7            Werder Bremen       26%          47%              27%
8        Borussia Dortmund       24%          48%              27%
9                Wolfsburg       30%          44%              26%
10           VfB Stuttgart       28%          46%              26%
11                Augsburg       28%          46%              26%
12     Borussia M.Gladbach       27%          47%              26%
13       

## 4.  Characteristics

In [43]:
import requests
from bs4 import BeautifulSoup
from pprint import pprint
from selenium import webdriver
from pprint import pprint
import pandas as pd
from time import sleep
from selenium.webdriver.common.action_chains import ActionChains


driver_path = 'C:\\SEONMIN\\chromedriver_win32\\chromedriver'
url = 'https://www.whoscored.com/Teams/7614/Show/Germany-RasenBallsport-Leipzig'

def get_character():
    print('RB Leipzig Characteristics')
    driver = webdriver.Chrome(driver_path)
    driver.get(url)
    
    strength = driver.find_element_by_xpath('//*[@id="layout-content-wrapper"]/div[2]/div[7]/div[3]/div[1]/div[1]').text
    print(strength)
    
    weakness = driver.find_element_by_xpath('//*[@id="layout-content-wrapper"]/div[2]/div[7]/div[3]/div[1]/div[2]').text
    print(weakness)
    
    style_of_Play = driver.find_element_by_xpath('//*[@id="layout-content-wrapper"]/div[2]/div[7]/div[3]/div[3]').text
    print(style_of_Play)

In [44]:
get_character()

RB Leipzig Characteristics
+ Strengths
Creating chances through individual skill
Very Strong
Aerial duels
Very Strong
Stealing the ball from the opposition
Very Strong
Creating scoring chances
Strong
Creating chances using through balls
Strong
Shooting from direct free kicks
Strong
Defending set pieces
Strong
- Weaknesses
Avoiding offside
Weak
Keeping possession of the ball
Weak
RasenBallsport Leipzig's Style of Play
Play with width
Attempt through balls often
Control the game in the opposition's half
Attacking down the left
Take a lot of shots
Play the offside trap
Aggressive
Rotate their first eleven
